In [44]:
import pandas_gbq
import folium
from folium.plugins import MarkerCluster

query_ocorrencias = """SELECT *
            FROM `rj-cor.adm_cor_comando.ocorrencias`
            WHERE status = 'Aberto'
            ORDER BY data_inicio DESC"""

# query_explicacao = """SELECT *
#             FROM `datario.adm_central_atendimento_1746.chamado`
#             LIMIT 20"""

df_ocorrencias = pandas_gbq.read_gbq(query_ocorrencias, project_id = "hackaton-fgv").dropna(subset=['latitude', 'longitude'])

# df_explicacao = pandas_gbq.read_gbq(query_explicacao, project_id = "hackaton-fgv")
# display(df_explicacao.head())

# Criando um mapa centrado no Rio de Janeiro
rio_map = folium.Map(location=[-22.9068, -43.1729], zoom_start=11)

# Adicionando um cluster de marcadores ao mapa
marker_cluster = MarkerCluster().add_to(rio_map)

# Iterando sobre as linhas do DataFrame para adicionar marcadores ao cluster
for index, row in df_ocorrencias.iterrows():
    if row['gravidade'] == 'Normal':
        popup_content = f"<div style='width: 200px;'><b>Local exato:</b><br>{row['descricao'].title()}</div>"
        folium.Marker(location=[row['latitude'], row['longitude']], popup=popup_content, 
                      icon=folium.Icon(color='green')).add_to(marker_cluster)
    elif row['gravidade'] == 'Média':
        popup_content = f"<div style='width: 200px;'><b>Local exato:</b><br>{row['descricao'].title()}</div>"
        folium.Marker(location=[row['latitude'], row['longitude']], popup=popup_content, 
                      icon=folium.Icon(color='orange')).add_to(rio_map)
    elif row['gravidade'] == 'Alta':
        popup_content = f"<div style='width: 200px;'><b>Local exato:</b><br>{row['descricao'].title()}</div>"
        folium.Marker(location=[row['latitude'], row['longitude']], popup=popup_content, 
                      icon=folium.Icon(color='red')).add_to(rio_map)
        
legend_html = """
<div style="position: fixed; bottom: 50px; right: 50px; z-index:9999; font-size:14px; background-color:rgba(255, 255, 255, 0.8); padding:10px; border-radius:10px; box-shadow:0px 0px 10px rgba(0, 0, 0, 0.2);">
    <b>Gravidade: </b><br>
    <i class="fa fa-circle" style="color:red"></i> Alta <br>
    <i class="fa fa-circle" style="color:orange"></i> Média<br>
    <i class="fa fa-circle" style="color:green"></i> Normal
</div>
"""

rio_map.get_root().html.add_child(folium.Element(legend_html))

# Exibindo o mapa
rio_map

Downloading: 100%|██████████|
